In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import os
import warnings
from scipy.optimize import minimize
warnings.filterwarnings('ignore')

from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit

try:
    import lightgbm as lgb
    HAS_LGB = True
except ImportError:
    HAS_LGB = False

# ============================================
# CONFIGURATION
# ============================================
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2
TRADING_DAYS_PER_YEAR = 252

if os.path.exists('/kaggle/input'):
    DATA_DIR = '/kaggle/input/hull-tactical-market-prediction'
else:
    DATA_DIR = '.'

# ============================================
# EVALUATION METRIC
# ============================================
def calculate_sharpe_ratio(positions, forward_returns, risk_free_rate):
    positions = np.array(positions)
    forward_returns = np.array(forward_returns)
    risk_free_rate = np.array(risk_free_rate)
    
    strategy_returns = risk_free_rate * (1 - positions) + positions * forward_returns
    strategy_excess_returns = strategy_returns - risk_free_rate
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    n = len(positions)
    strategy_mean_excess_return = strategy_excess_cumulative ** (1 / n) - 1
    strategy_std = strategy_returns.std()
    
    if strategy_std == 0:
        return -999
    
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR)
    strategy_volatility = strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100
    
    market_excess_returns = forward_returns - risk_free_rate
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = market_excess_cumulative ** (1 / n) - 1
    market_std = forward_returns.std()
    market_volatility = market_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100
    
    if market_volatility == 0:
        return -999
    
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2)
    vol_penalty = 1 + excess_vol
    
    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * TRADING_DAYS_PER_YEAR)
    return_penalty = 1 + (return_gap ** 2) / 100
    
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return adjusted_sharpe

# ============================================
# FEATURE ENGINEERING
# ============================================
class FeatureEngineer:
    def __init__(self):
        self.imputer = SimpleImputer(strategy='median')
        self.scaler = RobustScaler()
        self.feature_cols = None
        self.all_feature_cols = None
        
    def _add_engineered_features(self, df):
        df = df.copy()
        m_cols = [c for c in df.columns if c.startswith('M')]
        v_cols = [c for c in df.columns if c.startswith('V')]
        s_cols = [c for c in df.columns if c.startswith('S')]
        e_cols = [c for c in df.columns if c.startswith('E')]
        
        for prefix, cols in [('M', m_cols), ('V', v_cols), ('S', s_cols), ('E', e_cols)]:
            if cols:
                df[f'{prefix}_mean'] = df[cols].mean(axis=1)
                df[f'{prefix}_std'] = df[cols].std(axis=1)
        
        return df
        
    def fit(self, df, feature_cols):
        self.feature_cols = feature_cols
        df_eng = self._add_engineered_features(df)
        eng_cols = [c for c in df_eng.columns if c.endswith(('_mean', '_std'))]
        self.all_feature_cols = feature_cols + [c for c in eng_cols if c not in feature_cols]
        
        X = df_eng[self.all_feature_cols].values
        X_imputed = self.imputer.fit_transform(X)
        self.scaler.fit(X_imputed)
        return self
    
    def transform(self, df):
        df_eng = self._add_engineered_features(df)
        X = df_eng[self.all_feature_cols].values
        X_imputed = self.imputer.transform(X)
        X_scaled = self.scaler.transform(X_imputed)
        return X_scaled
    
    def fit_transform(self, df, feature_cols):
        self.fit(df, feature_cols)
        return self.transform(df)

# ============================================
# STRATEGY
# ============================================
class RobustStrategy:
    def __init__(self):
        self.sensitivity = 1500
        self.max_weight = 1.3
        self.min_weight = 0.5
        
    def predict_to_weight(self, predictions):
        predictions = np.array(predictions)
        scaled = predictions * self.sensitivity
        sigmoid_output = 1 / (1 + np.exp(-scaled))
        weights = self.min_weight + (self.max_weight - self.min_weight) * sigmoid_output
        weights = np.clip(weights, MIN_INVESTMENT, MAX_INVESTMENT)
        return weights
    
    def optimize_across_windows(self, all_predictions, all_returns, all_rf_rates):
        def objective(params):
            sensitivity, max_w, min_w = params
            if min_w >= max_w: return 1e9
            
            sharpes = []
            for preds, rets, rfs in zip(all_predictions, all_returns, all_rf_rates):
                scaled = preds * sensitivity
                sigmoid_output = 1 / (1 + np.exp(-scaled))
                weights = min_w + (max_w - min_w) * sigmoid_output
                weights = np.clip(weights, MIN_INVESTMENT, MAX_INVESTMENT)
                sharpe = calculate_sharpe_ratio(weights, rets, rfs)
                sharpes.append(sharpe)
            
            score = np.mean(sharpes) - 0.5 * np.std(sharpes)
            return -score

        x0 = [1500, 1.3, 0.5] 
        bounds = [(500, 5000), (1.0, 2.0), (0.0, 1.0)]
        result = minimize(objective, x0, method='Nelder-Mead', bounds=bounds, tol=1e-4)
        
        best_params = result.x
        self.sensitivity = best_params[0]
        self.max_weight = best_params[1]
        self.min_weight = best_params[2]
        
        return self

# ============================================
# ENSEMBLE MODEL
# ============================================
class EnsembleModel:
    def __init__(self):
        self.models = []
        self.weights = []
        
    def _optimize_weights(self, predictions_list, y_true):
        def objective(weights):
            weights = np.array(weights)
            weights = weights / np.sum(weights)
            final_pred = np.zeros_like(predictions_list[0])
            for i, w in enumerate(weights):
                final_pred += w * predictions_list[i]
            return -np.corrcoef(final_pred, y_true)[0, 1]

        n_models = len(predictions_list)
        init_weights = [1.0 / n_models] * n_models
        constraints = ({'type': 'eq', 'fun': lambda w: 1 - np.sum(w)})
        bounds = [(0, 1) for _ in range(n_models)]
        
        result = minimize(objective, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        return list(result.x / np.sum(result.x))

    def fit(self, X, y):
        split_idx = int(len(X) * 0.8)
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_train, y_val = y[:split_idx], y[split_idx:]
        
        if HAS_LGB:
            models_configs = [
                {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.03, 'num_leaves': 15,
                 'min_child_samples': 60, 'reg_alpha': 0.2, 'reg_lambda': 0.2,
                 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': 42},
                {'n_estimators': 250, 'max_depth': 5, 'learning_rate': 0.04, 'num_leaves': 20,
                 'min_child_samples': 50, 'reg_alpha': 0.15, 'reg_lambda': 0.15,
                 'subsample': 0.75, 'colsample_bytree': 0.75, 'random_state': 123},
                {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05, 'num_leaves': 25,
                 'min_child_samples': 40, 'reg_alpha': 0.1, 'reg_lambda': 0.1,
                 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': 456},
            ]
            self.models = [lgb.LGBMRegressor(**config, verbose=-1) for config in models_configs]
        else:
            self.models = [
                GradientBoostingRegressor(n_estimators=200, max_depth=4, learning_rate=0.05,
                                         min_samples_leaf=40, random_state=42),
                RandomForestRegressor(n_estimators=200, max_depth=6, min_samples_leaf=30,
                                     random_state=42, n_jobs=-1),
                Ridge(alpha=10.0)
            ]
        
        for model in self.models:
            model.fit(X_train, y_train)
            
        val_preds = [model.predict(X_val) for model in self.models]
        self.weights = self._optimize_weights(val_preds, y_val)
        
        for model in self.models:
            model.fit(X, y)
        
        return self
    
    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for model, weight in zip(self.models, self.weights):
            predictions += weight * model.predict(X)
        return predictions

# ============================================
# TRAINING
# ============================================
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

target_cols = ['forward_returns', 'risk_free_rate', 'market_forward_excess_returns']
id_col = 'date_id'
exclude_cols = [id_col] + target_cols

test_df_sample = pd.read_csv(f'{DATA_DIR}/test.csv', nrows=5)
test_cols = set(test_df_sample.columns)

feature_cols = [col for col in train_df.columns if col not in exclude_cols and col in test_cols]

# Filter training data
missing_by_date = train_df[feature_cols].isnull().sum(axis=1)
threshold = len(feature_cols) * 0.05
valid_mask = missing_by_date <= threshold
valid_start_idx = valid_mask.idxmax()
valid_start_date = train_df.loc[valid_start_idx, 'date_id']
train_clean = train_df[train_df['date_id'] >= valid_start_date].copy().reset_index(drop=True)

# Multi-window cross-validation
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

all_predictions = []
all_returns = []
all_rf_rates = []

for train_idx, val_idx in tscv.split(train_clean):
    train_data = train_clean.iloc[train_idx].copy()
    val_data = train_clean.iloc[val_idx].copy()
    
    fe_fold = FeatureEngineer()
    X_train = fe_fold.fit_transform(train_data, feature_cols)
    y_train = train_data['forward_returns'].values
    
    X_val = fe_fold.transform(val_data)
    
    model_fold = EnsembleModel()
    model_fold.fit(X_train, y_train)
    
    val_preds = model_fold.predict(X_val)
    
    all_predictions.append(val_preds)
    all_returns.append(val_data['forward_returns'].values)
    all_rf_rates.append(val_data['risk_free_rate'].values)

# Optimize strategy
strategy = RobustStrategy()
strategy.optimize_across_windows(all_predictions, all_returns, all_rf_rates)

# Final training
fe = FeatureEngineer()
X_full = fe.fit_transform(train_clean, feature_cols)
y_full = train_clean['forward_returns'].values

model = EnsembleModel()
model.fit(X_full, y_full)

# ============================================
# PREDICTION FUNCTION
# ============================================
def predict(test_batch: pl.DataFrame) -> pl.DataFrame:
    test_pd = test_batch.to_pandas()
    
    if 'date_id' in test_pd.columns:
        row_ids = test_pd['date_id'].values
    else:
        row_ids = test_pd.iloc[:, 0].values
    
    X_test = fe.transform(test_pd)
    predictions = model.predict(X_test)
    weights = strategy.predict_to_weight(predictions)
    weights = np.clip(weights, MIN_INVESTMENT, MAX_INVESTMENT)
    
    result = pl.DataFrame({
        'date_id': row_ids,
        'prediction': weights
    })
    
    return result

# ============================================
# KAGGLE INFERENCE SERVER
# ============================================
import kaggle_evaluation.core.templates
from kaggle_evaluation.default_gateway import DefaultGateway

class HullTacticalInferenceServer(kaggle_evaluation.core.templates.InferenceServer):
    def __init__(self):
        super().__init__(predict)
    
    def _get_gateway_for_test(self, data_paths=None, file_share_dir=None):
        return DefaultGateway(data_paths)

# ============================================
# RUN
# ============================================
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server = HullTacticalInferenceServer()
    inference_server.serve()
else:
    test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
    test_pl = pl.from_pandas(test_df)
    submission = predict(test_pl)
    submission_pd = submission.to_pandas()
    
    import pyarrow as pa
    import pyarrow.parquet as pq
    
    table = pa.Table.from_pandas(submission_pd, preserve_index=False)
    pq.write_table(table, 'submission.parquet')
    submission_pd.to_csv('submission.csv', index=False)
    
    print(f'Submission saved: {len(submission_pd)} predictions')
    print(f'Weight range: [{submission_pd["prediction"].min():.4f}, {submission_pd["prediction"].max():.4f}]')
